# Modelos linguísticos com n-gramas

Até o momento, usamos modelos probabilísticos para a probabilidade de encontrar uma palavra $w$ em algum documento dentro da coleção $c$, isto é: $P(w | c)$. Implicitamente, isso quer dizer que a *ordem* das palavras dentro de um documento não tem impacto em seu significado. Metaforicamente, é como se colocássemos todas as palavras em uma grande sacola, e por isso esse tipo de representação baseado na presença ou não de palavras é chamado de *bag-of-words*.

O modelo bag-of-words é eficaz para muitas aplicações, mas pode perder características importantes de uma palavra: por um lado, é pouco provável que um texto que mencione "ornitorrincos" e "cangurus" não se refira a ornitorrincos e cangurus; por outro lado, o texto: "ornitorrincos são mais perigosos que cangurus" é muito diferente de "cangurus são mais perigosos que ornitorrincos".

Uma maneira de criar modelos para a *ordem* em que as palavras aparecem em um texto é chamado de modelo linguístico gerador (ou generativo ou gerativo, dependendo da tradução que você adotar) (*generative linguistic model*). Nesse tipo de modelo, estimamos a probabilidade de encontrar a $n$-ésima palavra de uma sequência com base na palavra anterior, isto é:

$$
P(w_n | w_{n-1})
$$

Podemos fazer um pequeno modelo para a passagem:

    Passa um, passa dois, passa três

Nesse caso, nosso modelo nos dá probabilidades como:

* $P(\text{passa} | \text{um}) = 1$
* $P(\text{passa} | \text{dois}) = 1$
* $P(\text{dois} | \text{passa}) = 1/3$

Veja que essas probabilidades são estimadas por contagem em um conjunto de dados de treino!

## Exercício 1
**Objetivo: calcular probabilidades de um modelo linguístico baseado em uma frase**

Na passagem:

    Joana foi passear com alguns de seus sete cachorros durante uma tarde ensolarada, e se encontrou com uma amiga. Uma pessoa ali presente também parou para conversar com elas, e também pararam alguns outros cachorros para brincar com os de Joana.

Calcule:

* $P(\text{tarde} | \text{uma})$
    - <font color=red>1/3</font>


* $P(\text{alguns} | \text{com})$
    - <font color=red>1/4</font>


In [1]:
texto = 'Joana foi passear com alguns de seus sete cachorros durante uma tarde ensolarada, e se encontrou com uma amiga. Uma pessoa ali presente também parou para conversar com elas, e também pararam alguns outros cachorros para brincar com os de Joana.'
palavras = texto.upper().split()
print(len(palavras))
print(palavras.count('TARDE'), palavras.count('UMA'))
print(palavras.count('ALGUNS'), palavras.count('COM'))

41
1 3
2 4


## Exercício 2
**Objetivo: estimar um vocabulário à partir de dados**

Usando reviews do `IMDB`, monte um vocabulário de todas as palavras que podem ser usadas para gerar reviews de filmes. Use tokens também para representar os sinais de pontuação, isto é, ` "aqui, agora"` deve ser tokenizado como `['aqui', ',', 'agora']`. Para isso, complete o código abaixo trocando a expressão regular que aparece no `re.findall`.

In [4]:
import pandas as pd
import re
df = pd.read_csv('datasets/IMDB Dataset.csv')

In [134]:
vocab = set()
#REGEX = r'[a-z]+'
REGEX = r'[A-Za-z0-9]\w*|[^\w\s]'
#REGEX = r'\b\w+\b|[,.!]'
for text in df['review'][0:1000]:
    tokens = re.findall(REGEX, text.lower())
    vocab = vocab.union(set(tokens))
print(len(vocab))

17995


## Exercício 3
**Objetivo: estimar um modelo linguístico à partir de dados**

O código abaixo realiza a contagem de tokens que seguem outros tokens usando a base de dados IMDB, bem como a quantidade total de aparições de cada token. Interprete o código e, depois, execute. Com base no resultado dele, estime:

* A probabilidade de a palavra `under` ser seguida pela palavra `her`.
* A probabilidade da palavra `violence` aparecer no fim de uma frase.

In [135]:
model = {key: {'[TOTAL]':0} for key in vocab}

for text in df['review'][0:1000]:
    tokens = re.findall(REGEX, text.lower())
    for i in range(len(tokens)-1):
        wn = tokens[i+1]
        wn1 = tokens[i]
        if wn in model[wn1].keys():
            model[wn1][wn] += 1
        else:
            model[wn1][wn] = 1
            
        model[wn1]['[TOTAL]'] += 1

In [136]:
#A probabilidade de a palavra under ser seguida pela palavra her -> P(her|under)
print(model['under']['her']/model['under']['[TOTAL]'])

#A probabilidade da palavra violence aparecer no fim de uma frase. -> P(.|violence)
print(model['violence']['.']/model['violence']['[TOTAL]'])

0.03571428571428571
0.19642857142857142


## Exercício 4
**Objetivo: programar um sistema de sugestão de palavras**

Com base no modelo que foi estimado no exercício anterior, programe uma função que recebe uma palavra e retorna uma possível palavra seguinte. Caso a palavra usada como base não faça parte do vocabulário do modelo, ele deve retornar uma palavra aleatória de seu vocabulário. Use a funcionalidade de `np.random.choice` para fazer escolhas com probabilidades pré-definidas, como abaixo:

In [137]:
import numpy as np
print(np.random.choice(['um', 'dois', 'tres'], p=[0.5, 0.2, 0.3]))

um


In [182]:
def sugestao_de_palavras(token):
    if token in model:
        choices = [word for word in model[token] if word != '[TOTAL]']
        probs = [model[token][word]/model[token]['[TOTAL]'] for word in choices]
        return np.random.choice(choices, p=probs)

    else: 
        return np.random.choice(list(vocab))

In [184]:
sugestao_de_palavras('superior')

'film'

## Exercício 5
**Objetivo: fazer um gerador de textos**

Agora, podemos fazer um gerador de textos. Para isso, vamos gerar uma palavra, e então incorporá-la ao texto que temos, gerando palavras sucessivamente.

Complete a função `gerar` abaixo para que ela retorne uma string com `n_tokens` novas palavras geradas à partir da string de entrada `s`:

In [140]:
def gerar(s, n_tokens=10):
    REGEX = r'[A-Za-z0-9]\w*|[^\w\s]'
    tokens = re.findall(REGEX, s.lower())
    size = len(tokens)
    
    for n in range(n_tokens):
        tokens.append(sugestao_de_palavras(tokens[-1]))
    
    return ' '.join(tokens[size:])

In [142]:
s = 'After watching the movie, I'
gerar(s)

'have bought this film out what kept that day australia'

## Exercício 6
**Objetivo: refletir sobre a aplicação de n-gramas em modelos linguísticos**

Até o momento, usamos apenas uma palavra do passado para determinar o contexto que gera uma nova palavra. Poderíamos, porém, usar um número arbitrário $N$ de palavras anteriores, isto é:

$$
P(w_n | w_{n-1}, w_{n-2}, ... , w_{n-N})
$$

Podemos fazer um pequeno modelo para a passagem:

    Passa um, passa dois, passa três

Nesse caso, um modelo como o que temos feito até o momento nos daria probabilidades como:

* $P(\text{passa} | \text{um}) = 1$
* $P(\text{passa} | \text{dois}) = 1$
* $P(\text{dois} | \text{passa}) = 1/3$

Um modelo com $N=2$, por sua vez, teria:

* $P(\text{passa} | \text{um, passa}) = 1$
* $P(\text{passa} | \text{dois, passa}) = 1$
* $P(\text{dois} | \text{passa, um}) = 1$

Neste momento, aparece um novo elemento importante para nós. Nossa coleção não está mais sendo descrita por palavras, e sim por conjuntos de palavras ordenadas! Esses conjuntos são chamados de n-gramas.

A função abaixo permite extrair n-gramas de uma lista de tokens:

In [143]:
def get_ngrams(tokens, N):
    ngrams = [tuple(tokens[i:i+N]) for i in range(len(tokens)-N+1)]
    return ngrams

print(get_ngrams("isto é um teste e n-gramas podem ser legais".split(), N=3))

[('isto', 'é', 'um'), ('é', 'um', 'teste'), ('um', 'teste', 'e'), ('teste', 'e', 'n-gramas'), ('e', 'n-gramas', 'podem'), ('n-gramas', 'podem', 'ser'), ('podem', 'ser', 'legais')]


1. Qual é a vantagem em usar n-gramas com valores de $N$ maiores?
    - <font color=red>Compreender o contexto da frase.</font>

2. Quando $N$ aumenta, qual é a probabilidade de encontrarmos ocorrências de n-gramas em mais de um documento?
    - <font color=red>Com N muito algo os n-gramas ficam cada vez mais específicos, diminuindo a chance de encontrar a ocorrência de n-gramas iguais.</font>

3. Como essa propriedade poderia ser utilizada para detectar cópias em trabalhos escritos? 
    - <font color=red>Sim!</font>

## Exercício 7
**Objetivo: usar n-gramas para melhorar nosso modelo linguístico**

O código abaixo mostra um modelo linguístico para fazer predições de palavras com base nas $N$ palavras anteriores. O código, inicialmente, usa $N=2$.

Implemente um gerador de textos baseado neste novo modelo linguístico.

Caso as duas palavras que estejam no fim do texto recebido como entrada não estejam no banco de dados deste modelo, o sistema deve "chamar" o modelo anterior (baseado em somente uma palavra).

In [201]:
def get_ngrams_and_res(tokens, N):
    ngrams = [tuple(tokens[i:i+N]) for i in range(len(tokens)-N)]
    res = [tokens[i+N] for i in range(len(tokens)-N)]
    return ngrams, res

# Criar vocabulário
vocab = set()
all_ngrams = []
all_res = []
for text in df['review'][0:5000]:
    tokens = re.findall(REGEX, text.lower())
    n_grams,res = get_ngrams_and_res(tokens, 2)
    all_ngrams += n_grams
    all_res += res
    vocab = vocab.union(set(n_grams))

# Criar modelo
model_n2 = {key: {'[TOTAL]':0} for key in vocab}

for i in range(len(all_ngrams)):
    wn = all_res[i]
    wn1 = all_ngrams[i]
    if wn in model_n2[wn1].keys():
        model_n2[wn1][wn] += 1
    else:
        model_n2[wn1][wn] = 1
        
    model_n2[wn1]['[TOTAL]'] += 1

In [202]:
def sugestao_de_palavras_n2(token):
    if token in model_n2:
        choices = [word for word in model_n2[token] if word != '[TOTAL]']
        probs = [model_n2[token][word]/model_n2[token]['[TOTAL]'] for word in choices]
        return np.random.choice(choices, p=probs)

    else: 
        return sugestao_de_palavras(token[-1])

In [208]:
def gerar_n2(s, n_tokens):
    REGEX = r'[A-Za-z0-9]\w*|[^\w\s]'
    tokens = re.findall(REGEX, s.lower())
    size = len(tokens)
    
#     for n in range(n_tokens):
#         if (tokens[-2], tokens[-1]) not in model_n2:
#             print(tokens[-2], tokens[-1])
#             tokens.append(sugestao_de_palavras(model_n2, (tokens[-2], tokens[-1])))
#         else:
#             tokens.append(sugestao_de_palavras(model, tokens[-1]))
    
    for n in range(n_tokens):
        tokens.append(sugestao_de_palavras_n2((tokens[-2], tokens[-1])))
    
    return ' '.join(tokens)

gerar_n2('dumb jokes', 20)

'dumb jokes and gags that get a region 5 chinese dvd of vampyr - der traum des allen grey , the silver'

## Exercício 8
**Objetivo: testar o gerador de textos em uma situação real**

Escreva um trecho de um pequeno review de filmes em inglês. Use seu gerador para completar seu review. Como você avalia o desempenho dele?